In [1]:
from dotenv import load_dotenv
from kaggle.api.kaggle_api_extended import KaggleApi

load_dotenv()

api = KaggleApi()
api.authenticate()
api.dataset_download_files('ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training', path='.', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('dirty_cafe_sales.csv')
rng = np.random.default_rng(seed=42)
print(df.shape)
df.head()

(10000, 8)


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [3]:
df.replace("UNKNOWN", np.nan, inplace=True)
df.replace("ERROR", np.nan, inplace=True)
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,NaN,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,NaN,NaN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [4]:
df["Price Per Unit"] = df["Price Per Unit"].astype(float,errors='ignore')
df["Total Spent"] = df["Total Spent"].astype(float,errors='ignore')
df["Quantity"] = df["Quantity"].astype(float,errors='ignore')
df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")

df.dtypes

Transaction ID              object
Item                        object
Quantity                   float64
Price Per Unit             float64
Total Spent                float64
Payment Method              object
Location                    object
Transaction Date    datetime64[ns]
dtype: object

In [5]:
df.isna().sum()

Transaction ID         0
Item                 969
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

In [6]:
subsection = df[["Item", "Price Per Unit"]]
subsection = df[df["Item"].notna() & df["Price Per Unit"].notna()]

item_price_map = {}
subsection.apply(lambda row: item_price_map.update({row['Item']: row['Price Per Unit']}), axis=1)
item_price_map

{'Coffee': 2.0,
 'Cake': 3.0,
 'Cookie': 1.0,
 'Salad': 5.0,
 'Smoothie': 4.0,
 'Sandwich': 4.0,
 'Juice': 3.0,
 'Tea': 1.5}

In [7]:
sandwich_smoothie_probs = df[(df["Item"] == "Sandwich") | (df["Item"] == "Smoothie")]["Item"].value_counts(normalize=True)

In [8]:
for row in df.itertuples():
    item = row.Item
    quantity = row.Quantity
    price_per_unit = row._4
    total_spent = row._5


    # Price Per Unit inference
    if pd.isna(price_per_unit) and pd.notna(item):
        df.at[row.Index, "Price Per Unit"] = item_price_map[item]
        price_per_unit = df.at[row.Index, "Price Per Unit"]
    elif pd.isna(price_per_unit) and pd.notna(total_spent) and pd.notna(quantity) and quantity >= 1:
        df.at[row.Index, "Price Per Unit"] = total_spent / quantity
        price_per_unit = df.at[row.Index, "Price Per Unit"]

    # Item inference
    if pd.isna(item) and pd.notna(price_per_unit):
        for key, value in item_price_map.items():
            if value == price_per_unit:
                if key == "Sandwich" or key == "Smoothie":
                    df.at[row.Index, "Item"] = rng.choice(sandwich_smoothie_probs.index, p=sandwich_smoothie_probs.values)
                else:
                    df.at[row.Index, "Item"] = key
                break

    # Total Spent inference
    if pd.isna(total_spent) and pd.notna(price_per_unit) and pd.notna(quantity):
        df.at[row.Index, "Total Spent"] = price_per_unit * quantity
        total_spent = df.at[row.Index, "Total Spent"]


    # Quantity inference
    if pd.isna(quantity) and pd.notna(price_per_unit) and pd.notna(total_spent):
        df.at[row.Index, "Quantity"] = total_spent / price_per_unit
        quantity = df.at[row.Index, "Quantity"]


In [9]:
print(df.isna().sum())

Transaction ID         0
Item                   6
Quantity              23
Price Per Unit         6
Total Spent           23
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64


In [10]:
def probability_fill(col,probs_df):
    df[col] = df[col].apply(lambda val: rng.choice(probs_df.index,p=probs_df.values) if pd.isna(val) else val)


location_probs = df["Location"].value_counts(normalize=True)
payment_method_probs = df["Payment Method"].value_counts(normalize=True)


probability_fill("Location", location_probs)
probability_fill("Payment Method", payment_method_probs )

In [11]:
df["Transaction Date"] = df["Transaction Date"].sort_index().ffill().reset_index(drop=True)

In [12]:
df[df.isna().any(axis=1)]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
236,TXN_8562645,Salad,NaN,5.0,NaN,Cash,In-store,2023-05-18
278,TXN_3229409,Juice,NaN,3.0,NaN,Cash,Takeaway,2023-04-15
641,TXN_2962976,Juice,NaN,3.0,NaN,Credit Card,In-store,2023-03-17
738,TXN_8696094,Sandwich,NaN,4.0,NaN,Credit Card,Takeaway,2023-05-14
1761,TXN_3611851,NaN,4.0,NaN,NaN,Credit Card,Takeaway,2023-02-09
2289,TXN_7524977,NaN,4.0,NaN,NaN,Credit Card,In-store,2023-12-09
2796,TXN_9188692,Cake,NaN,3.0,NaN,Credit Card,Takeaway,2023-12-01
3203,TXN_4565754,Smoothie,NaN,4.0,NaN,Digital Wallet,Takeaway,2023-10-06
3224,TXN_6297232,Coffee,NaN,2.0,NaN,Credit Card,Takeaway,2023-04-07
3401,TXN_3251829,Tea,NaN,1.5,NaN,Digital Wallet,In-store,2023-07-25


In [13]:
df.dropna(inplace=True)
df.shape

(9974, 8)

In [14]:
df.to_csv('cleaned_cafe_sales.csv', index=False)